# Introduction to distributions and basic sampling in CUQIpy

This notebook describes basic usage of distributions in CUQIpy including visualizing their probability density function (PDF) and cumulative distribution function (CDF), and generating samples. Then conditional distributions are demonstrated along with the creation of user-defined distributions.


## Learning objectives of this notebook:
Going through this notebook, you will see how to:
- Set up random variables following univariate and multivariate distributions in CUQIpy.
- Generate samples from distributions and use CUQIpy tools to inspect visually.
- Set up conditional distributions in CUQIpy - simple and using lambda functions.
- ★ Create a user-defined distribution from a logpdf function.


## Table of contents: 
* [1. Normal distribution (univariate)](#Normal)
* [2. Multivariate distributions](#Multivariate)
* [3. Conditional distributions ](#Conditional)
* [4. User-defined distributions ★](#Userdefined)


★ Indicates optional section.

## References
[1] *Bardsley, Johnathan. 2018. Computational Uncertainty Quantification for Inverse Problems. SIAM, Society for Industrial and Applied Mathematics.*




First we need to import any Python packages needed, here NumPy for array computations and Matplotlib for plotting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

We import CUQIpy. In the other notebooks we may import upfront the specific tools we need, like `from cuqi.distribution import Gaussian` to get the Gaussian distribution from CUQIpy's distribution module. We now simply import the complete package and then specify the complete name such as `cuqi.distribution.Gaussian` when using it. Both approaches are fine, each with pros and cons.

In [ ]:
import cuqi

## 1. Normal distribution  (univariate)  <a class="anchor" id="Normal"></a> 

The first thing we can do is to define a simple normal distribution of a single variable, e.g.,

$$ x \sim \mathrm{Normal}(0,1^2) $$

This is done using the following syntax:

In [ ]:
x = cuqi.distribution.Normal(mean=0, std=1)

More information on the distribution can be found in the [CUQIpy API documentation](https://cuqi-dtu.github.io/CUQIpy/api/index.html).

Once created, we can print the distribution object, and its dimension and name:

In [ ]:
print(x)
print(x.dim)
print(x.name)

**Note about the distribution name:**

- The distribution name is inferred automatically from the assignment statement `x = cuqi.distribution.Normal(mean=0, std=1)`. 
- Since the distribution was assigned to the variable `x`, by default, the distribution name will be `x`.
- One can specify a different name by explicitly passing the `name` argument, e.g. `x = cuqi.distribution.Normal(mean=0, std=1, name='x1')`.


We can query other information about this distribution such as its mean and standard deviation:

In [ ]:
print(x.mean)
print(x.std)

Distributions in CUQIpy have commonly used methods that one might expect like *pdf*, *logpdf*, *cdf*, etc. For example we can evaluate the CDF at 0, which should be 0.5, since the pdf is symmetric about 0:

In [ ]:
x.cdf(0)

We can evaluate and plot the CDF on an interval by evaluating the CDF on a grid:

In [ ]:
grid = np.linspace(-10, 10, 1001)
cdf_vals = np.zeros(grid.shape)
for k in range(len(grid)):
    cdf_vals[k] = x.cdf(grid[k])
plt.plot(grid, cdf_vals)

Alternative more compact form using python's list comprehension:

In [ ]:
plt.plot(grid, [x.cdf(node_k) for node_k in grid])

CUQIpy distributions also have `sample` method which returns one or more samples from the distribution as a `CUQIarray`:

In [ ]:
x.sample()

By default a single sample is returned. More samples can easily be requested:

In [ ]:
x_samples = x.sample(10000)
type(x_samples)

When more than one sample is generated, a CUQIpy `Samples` object is returned. This is essentially an array in which each column contains one sample, and further equipped with a number of methods for computing samples statistics, diagnostics and plotting.

We can look at all the plotting methods available for the `Samples` object:

In [ ]:
# A list of all plotting methods
[method for method in dir(x_samples) if method.startswith('plot')] 

For example, one can make a "chain plot", i.e., the sampled values of selected parameter(s) of interest. Here we have a single parameter and with Python being zero-indexed, we specify this parameter, the 0-th element of `x`, as follows:

In [ ]:
x_samples.plot_chain(0)

Another possibility is a histogram of the parameter chain: (The keyword arguments are passed directly to the underlying matplotlib `hist` function for full control). Again, we specify 0 as the element to look at the chain for:

In [ ]:
x_samples.hist_chain(0, bins=100, density=True)

CUQIpy has integrated support for common statistical plots with the [ArviZ library](https://arviz-devs.github.io/arviz/), for example a "trace plot" combines the previous two plots, where the histogram is replaced by a kernel density estimate (KDE).

In [ ]:
x_samples.plot_trace()

#### Try yourself (optional):  
 - Create a new random variable `r` following a normal distribution with mean 2 and standard deviation 3.
 - Generate 100 samples and display a histogram.
 - Compare with the theoretical distribution by plotting the probability density function of `r` on top of the histogram.
 - Increase the number of samples and (hopefully) see the histogram approach the theoretical PDF.

In [ ]:
# Type code here:



## 2. Multivariate distributions <a class="anchor" id="Multivariate"></a> 

CUQIpy currently implements a number of multivariate distributions in the `cuqi.distribution` module:

In [ ]:
[dist for dist in dir(cuqi.distribution) if not dist.startswith('_')]

and more can easily be added when needed.


As a demonstration, we specify here a 4-element random variable `y` following a Gaussian distribution with independent elements:

$$\mathbf{y} \sim \mathrm{Gaussian}(\mu,\mathrm{diag}(\sigma^2)) \quad \text{for} \quad \mu = [5, 3, 1, 0]^T \quad \text{and} \quad \sigma = [1,2,3, 0.5]$$

In [ ]:
true_mu = np.array([5, 3, 1, 0])
true_sigma = np.array([1, 2, 3, 0.5])
y = cuqi.distribution.Gaussian(mean=true_mu, cov=true_sigma**2)

As before, we can take a look at the distribution by printing it and its dimension:

In [ ]:
print(y)
print(y.dim)
print(y.name)

as well as its mean

In [ ]:
print(y.mean)

and its diagonal covariance matrix (represented as a 1D array of the diagonal values):

In [ ]:
print(y.cov)

We generate a single sample, which is represented by a 4-element `CUQIarray`:

In [ ]:
y.sample()

If we ask for more than one sample, say 1000, we get a `Samples` object with 1000 columns each holding a 4-element sample:

In [ ]:
y_samples = y.sample(1000)
print(y_samples)
y_samples.shape

We can plot chains of a few of these variable samples, here we pick element 2 and 0:

In [ ]:
y_samples.plot_chain([2, 0])

We can as well plot a few samples of `y`, each containing 4 elements:

In [ ]:
y_samples.plot(marker='o');

By default 5 random samples are plotted, but we can also specify indices of specific samples we wish to plot, like the 1st and 3rd samples:

In [ ]:
y_samples.plot([1, 3], marker='o');

Another option for inspecting samples is a "violin plot", which displays the median as a white circle, the interquartile range, along with the density/histogram on either side:

In [ ]:
y_samples.plot_violin()

We can also plot the sample mean and compare with the true mean of the distribution:

In [ ]:
y_samples.plot_mean(label="Sample mean")
plt.plot(y.mean, 'o', label="Distribution mean")
plt.legend()

and sample standard deviation along with the true standard deviations of the distribution which we obtain as the square-root of the diagonal of the covariance matrix:

In [ ]:
y_samples.plot_std(label="Sample std")
plt.plot(np.sqrt(y.cov), 'o', label="Distribution std")
plt.legend()

#### Try yourself (optional):  
 - Plot mean with 95% credibility interval, hint: `help(y_samples.plot_ci)`.
 - When plotting credibility interval with ``plot_ci``, plot also the true mean using the `exact` keyword argument.
 - Reduce and increase the number of samples and study the effect on the mean and credibility interval.
 - Try also 50% and 99% credibility intervals.

In [ ]:
# Type code here:


## 3. Conditional distributions <a class="anchor" id="Conditional"></a> 

CUQIpy also support conditional probability distributions, which are probability distributions that are defined conditionally on the value of one or more other random variables. 

In CUQIpy, defining conditional distributions is simple. Assume we are interested in defining the Normal distribution conditioned on a random variable representing the standard deviation, e.g.

$$ z \mid \mathrm{std} \sim \mathrm{Normal}(0,\mathrm{std}^2) $$

This can simply be achieved by *omitting* the keyword argument for the standard deviation, when defining the distribution, as shown in the following code

In [ ]:
z = cuqi.distribution.Normal(mean=0)

Printing it will tell us that the variable `std` has not been specified, i.e., it is a *conditioning variable*:

In [ ]:
print(z)

Because $z$ is a conditional distribution, we cannot evaluate the logpdf or sample it directly without specifying the value of the conditioning variable (the standard deviation in this case). Hence this code will fail to run:

In [ ]:
# This code will give an error so we wrap it in a try/except block and print the error
try:
    z.sample()
except Exception as e:
    print(e)

To specify the conditioning variable we use the "call" syntax, i.e., `z(std=2)` to set the value of the standard deviation in the conditional distribution as shown below.

In [ ]:
z(std=2).sample()

In fact, conditioning creates a new *unconditional* distribution. Here printing reveals that it does not have any conditioning variables:

In [ ]:
z_given_std = z(std=2)
print(z_given_std)

We expect we can then sample it directly, which is confirmed:

In [ ]:
z_given_std.sample()

In general, one may need more flexibility than simply conditioning directly on the attributes of the distribution. Let us assume we want to define a distribution that is conditional on the variance - denoted $d$ - rather than the standard deviation of the normal distribution, i.e.

$$ w \mid d \sim \mathrm{Normal}(0,d). $$

In CUQIpy this can be achieved through *lambda* functions. A lambda function is the Python equivalent of a MATLAB anonymous function, i.e. a function defined in a single line with the following syntax for an example function the simply sums two input arguments:

In [ ]:
myfun = lambda v1, v2: v1+v2

In [ ]:
myfun(5,7)

We can pass a lambda function directly as an argument to a CUQIpy distribution, e.g.,

In [ ]:
w = cuqi.distribution.Normal(mean=0, std=lambda d: np.sqrt(d))
print(w)

where we see that `d` is now the conditioning variable instead of `std` as before.

We can then pass a value for `d` to condition on, which allows us to sample from the now fully specified distribution:

In [ ]:
w(d=2).sample()

What actually happens behind the scenes is that writing `w(d=2)` defined a new CUQIpy distribution where the standard deviation is set by evaluating the lambda function. This can be seen by storing the new distribution as follows.

In [ ]:
w_given_d = w(d=2)
w_given_d.std

This framework allows for a lot of flexibility in defining conditional distributions. For example we can define lambda functions for all attributes (here the mean and the standard deviation):

In [ ]:
#Functions for mean and std with various (shared) inputs
mean = lambda sigma, gamma: sigma+gamma
std  = lambda delta, gamma: np.sqrt(delta+gamma)

u = cuqi.distribution.Normal(mean, std)
print(u)

The three variable names `sigma`, `gamma` and `delta` used to define the two lambda functions for the mean and standard deviation are now the conditioning variables of the conditional distribution `u`.

By providing values for all three variables we obtain a fully specified distribution

In [ ]:
u_given_all = u(sigma=3, delta=5, gamma=-2)
print(u_given_all)

that we can sample:

In [ ]:
u_given_all.sample()

Conditional distributions play a major role when specifying Bayesian inverse problems in CUQIpy and in particular those problems that include Bayesian hierarchical models where some random variables depend on other random variables. We revisit this in later notebooks.

## 4. User-defined distributions ★ <a class="anchor" id="Userdefined"></a> 

In addition to the distributions provided by CUQIpy, there is also the possibility for users to specify new distributions. One option is to write their own class in the same style as existing distributions such as the Cauchy distribution (see code here: [Cauchy](https://github.com/CUQI-DTU/CUQIpy/blob/main/cuqi/distribution/_cauchy.py)).

Another option is to specify a user-defined distribution, which is convenient if one, for example, only wishes to evaluate the logpdf.

The example below demonstrates how to manually specify a normal distribution through a lambda function for the logpdf and compare it to the normal distribution defined in the beginning of this notebook.

We specify variables for the mean and the standard deviation and specify the lambda function for the logpdf. 

In [ ]:
mu1 = 0
std1 = 1

logpdf_func = lambda xx: -np.log(std1*np.sqrt(2*np.pi))-0.5*((xx-mu1)/std1)**2

To set up the user-defined distribution, we need to specify the logpdf as well as its dimension (number of variables) since that cannot be automatically inferred from the lambda function:

In [ ]:
x_user = cuqi.distribution.UserDefinedDistribution(dim=1, logpdf_func=logpdf_func)

We can now evalute the logpdf, as well as the pdf:

In [ ]:
print(x_user.logpdf(0))
print(x_user.pdf(0))

We can compare this with the normal distribution from the beginning of the notebook and observe that their pdfs agree:

In [ ]:
plt.plot(grid, [x.pdf(node_k) for node_k in grid], label='CUQIpy Normal')
plt.plot(grid, [x_user.pdf(node_k) for node_k in grid], '--', label='User-defined Normal')
plt.legend()

We cannot sample the user-defined distribution because we have only provided the logpdf:

In [ ]:
try:
    x_user.sample()
except Exception as e:
    print(e)

We can equip the user-defined distribution with a sample_func which specified how to sample (it is up to the user to ensure consistency between logpdf and sample_func):

In [ ]:
x_user.sample_func = lambda : np.array(mu1 + std1*np.random.randn())

In [ ]:
x_user.sample()

We can compare the samples obtained from the original normal distribution and the user-defined:

In [ ]:
x_samples = x.sample(10000)

In [ ]:
x_user_samples = x_user.sample(10000)

We plot their histograms and note that they appear similar:

In [ ]:
x_samples.hist_chain(0,bins=100)
x_user_samples.hist_chain(0,bins=100)
plt.legend(['CUQIpy Normal', 'User-defined Normal'])